# Tutorial 6: Large Language Models

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: [Gabriel Iturra](https://giturra.cl/)


### Objetivos del Tutorial

- Que es Google Colab.
- Que es Python y como descargarlo.
- Entender como trabajar con archivos en Python, tanto de su versión nativa como con ```Pandas```.
- Entender como trabajar texto desde ```Pandas```.
- Entender como utilizar expresiones regulares.